<a href="https://colab.research.google.com/github/helmieresearch/helmieresearch/blob/main/Momentum_Ranking_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Install packages

In [1]:
!pip install yfinance

In [2]:
!pip install yahooquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 1.7 MB/s eta 0:00:00


In [3]:
!pip install stockstats

###Import data

In [4]:
%matplotlib inline
import datetime
from datetime import date
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#import mpld3
#mpld3.enable_notebook()
"""
Settings
"""
portfolio_value = 2000.00
perf_last_month = 2000.00
start = "2020-01-01"
end = "2023-08-02"
ranking_day = "2023-07-31"

# Storing today's date into a variable
today = date.today()
# Storing the specific date
trading_day = today
# Calculate percent difference since last month
perf_pct = (portfolio_value / perf_last_month) - 1
# Print performance, format as percent with two decimals.
print("{} - Last Month Result: {:.2%}".format(trading_day, perf_pct))
# Remember today's portfolio value for next month's calculation
#perf_last_month = portfolio_value

2023-07-31 - Last Month Result: 0.00%


In [5]:

now = datetime.datetime.now().strftime("%Y-%m-%d")
data = yf.Ticker("^J203.JO")
index = data.history(start=start,  end=end)
resetindex = index.reset_index(inplace = True)

index['Date'] = pd.to_datetime(index['Date']).dt.strftime('%Y-%m-%d')

index = index.drop(columns=(['Open', 'High','Low','Volume','Dividends','Stock Splits']))

index.set_index("Date", inplace=True)

index.to_csv('jse_index.csv')

In [6]:
#Import required libraries
import pandas as pd
import json
import requests
import glob
import os
from yahooquery import Ticker
from datetime import datetime

params = {'access_key': 'e52cf3b93696352e880916f8c8adbf0c',
          'limit': 350}
api_result = requests.get('http://api.marketstack.com/v1/exchanges/XJSE/tickers', params)
api_response = api_result.json()
print(f"Exchange Name = {api_response['data']['name']}")
for ticker in api_response['data']['tickers']:
  print(f"{ticker['name']}: {ticker['symbol']}")


# Serializing json
json_object = json.dumps(api_response, indent=4)
# Writing to sample.json
with open("jse.json", "w") as outfile:
    outfile.write(json_object)

# Opening JSON file
with open('jse.json', 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)
df1 = pd.json_normalize(json_object["data"]["tickers"])

# dropping null value columns to avoid errors
# new data frame with split value columns
new = df1["symbol"].str.split(".", n = 1, expand = True)
# making separate first name column from new data frame
df1["ticker"]= new[0]
# Dropping old Name columns
df1.drop(columns =["has_intraday","has_eod"], inplace = True)
new.columns = new.columns.astype(str)
new.columns.values[1] = "tickers"
new['0'] + '.' + new['tickers']
new.replace("XJSE", "JO", inplace=True)

result = pd.concat([df1, new], axis=1, join='inner')
df2=result['0'] + '.' + result['tickers']
final = pd.concat([df1, df2], axis=1, join='inner')
final.columns.values[3] = "tickers"
final.columns = final.columns.astype(str)
#Dropping old Name columns
final.drop(columns =["name", "symbol","ticker"], inplace = True)
# Converting a specific Dataframe
# column to list using Series.tolist()
xjse = final["tickers"].tolist()
print("Converting tickers to list...")
print("Output dataframe..." )
print("Writing csv file..." )
print("Saving JSE data to disc...")
print("Saving csv files for each stock to disc...")
symbols = xjse
tickers = Ticker(symbols, asynchronous=True)
data = tickers.history(start= start, end=end, interval='1d')
data= data.drop(['adjclose','dividends','splits'],axis=1)


#Convert dataframe to csv and save to disc
final.to_csv("tickers.csv")
data.to_csv("jse_stocks.csv")

resetindex = data.reset_index(inplace = True)

for i, g in data.groupby('symbol'):
    g.to_csv('/content/Stocks/{}.csv'.format(i), header=True, index_label=False)

# merging the files
joined_files = os.path.join("/content/Stocks/", "*.csv")

# A list of all joined files is returned
joined_list = glob.glob(joined_files)

# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# new dataframe with different column order
df_new = df.iloc[:, [1, 2, 3, 4, 5, 6, 0]]

Tickers = df_new["symbol"].tolist()
mylist = list(dict.fromkeys(Tickers))
df = pd.DataFrame (mylist, columns = ['tickers'])
df.to_csv("/content/tickers.csv")

#Change date format to Year-Month-Day
df_new['date'] =  pd.to_datetime(df_new['date']).dt.strftime('%Y-%m-%d')

df_new.set_index('date', inplace=True)

for i, g in df_new.groupby('symbol'):
    g.to_csv('/content/Stocks_final3/{}.csv'.format(i), header=True, index_label=False)

Exchange Name = Johannesburg Stock Exchange
PROSUS N.V.: PRX.XJSE
ANHEUSER-BUSCH INBEV SA NV: ANH.XJSE
BHP GROUP PLC: BHP.XJSE
BRITISH AMERICAN TOB PLC: BTI.XJSE
NASPERS LTD -N-: NPN.XJSE
COMPAGNIE FIN RICHEMONT: CFR.XJSE
GLENCORE PLC: GLN.XJSE
ANGLO AMERICAN PLC: AGL.XJSE
FIRSTRAND LTD: FSR.XJSE
ANGLO AMERICAN PLAT LTD: AMS.XJSE
STANDARD BANK GROUP LTD: SBK.XJSE
VODACOM GROUP LTD: VOD.XJSE
MONDI PLC: MNP.XJSE
CAPITEC BANK HLDGS LTD: CPI.XJSE
BEE - SASOL LIMITED: SOLBE1.XJSE
SANLAM LIMITED: SLM.XJSE
MTN GROUP LTD: MTN.XJSE
ANGLOGOLD ASHANTI LTD: ANG.XJSE
SASOL LIMITED: SOL.XJSE
ABSA GROUP LIMITED: ABG.XJSE
SOUTH32 LIMITED: S32.XJSE
RMB HOLDINGS LTD: RMH.XJSE
IMPALA PLATINUM HLGS LTD: IMP.XJSE
BID CORPORATION LTD: BID.XJSE
REMGRO LTD: REM.XJSE
KUMBA IRON ORE LTD: KIO.XJSE
NEDBANK GROUP LTD: NED.XJSE
SIBANYE STILLWATER LTD: SSW.XJSE
GOLD FIELDS LTD: GFI.XJSE
INVESTEC LTD: INL.XJSE
INVESTEC PLC: INP.XJSE
OLD MUTUAL LIMITED: OMU.XJSE
NEPI ROCKCASTLE PLC: NRP.XJSE
DISCOVERY LTD: DSY.XJSE
SH

In [7]:
symbols = xjse
tickers = Ticker(symbols, asynchronous=True)
data = tickers.history(start=start, end=end, interval='1d')
data= data.drop(['adjclose','dividends','splits'],axis=1)

resetindex = data.reset_index(inplace = True)

# new dataframe with different column order
df_new = data.iloc[:, [1, 2, 3, 4, 5, 6, 0]]

Tickers = df_new["symbol"].tolist()
mylist = list(dict.fromkeys(Tickers))
df = pd.DataFrame (mylist, columns = ['tickers'])
df.columns = df.iloc[1]

df.to_csv("/content/Stocks_final3/tickers.csv")

from pathlib import Path
import pandas as pd
import numpy as np
from scipy import stats

csv_folder = Path('/content/Stocks_final3')

for file in csv_folder.glob('*.JO.csv'): #create iteratable & iterate on it.
    mydata = pd.read_csv(file)
    mydata.reset_index(inplace = True)
    mydata = mydata.rename(columns={'index': 'date'}, index=None)
    mydata['date'] =  pd.to_datetime(mydata['date']).dt.strftime('%Y-%m-%d')
    mydata.set_index('date', inplace=True)
    df =mydata[["open","high","low","close","volume"]]
    no_outlier_prices = df[(np.abs(stats.zscore(df)) <3).all(axis=1)]
    new_file_name = file.parent.joinpath(f"{file.stem}.csv")
    no_outlier_prices.to_csv(new_file_name) #, index=None, encoding='utf-8-sig', decimal=',')

###Ranking list to construct initial portfolio

In [8]:
%matplotlib inline
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (12, 9) # (w, h)
plt.ioff()

tickers = pd.read_csv('/content/Stocks_final3/tickers.csv', header=None)[1].tolist()
stocks = (
    (pd.concat(
        [pd.read_csv(f"/content/Stocks_final3/{ticker}.csv",parse_dates=None, index_col=0)[
        'close'].rename(ticker)
        for ticker in tickers],
        axis=1,
        sort=True)
    )
)
stocks = stocks.loc[:,~stocks.columns.duplicated()]

from scipy.stats import linregress
def momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    return ((1 + slope) ** 252) * (rvalue ** 2)  # annualize slope and multiply by R^2

momentums = stocks.copy(deep=True)
for ticker in tickers:
    momentums[ticker] = stocks[ticker].rolling(90).apply(momentum, raw=False)

result = momentums.sort_values(by =ranking_day, axis = 1, ascending =False)
result.reset_index(inplace=True)
res=result[~(result['date'] < ranking_day)].dropna(axis=1)

mes = res.reindex(columns = res.columns.tolist()
                                  + ['empty'])

df = mes.set_index(['empty']).stack().reset_index(name='index').rename(columns={'level_2':'date'})

df.columns = df.iloc[0]

df.index = list(df.index)

df.columns.values[0] = "empty"
df.columns.values[1] = "symbol"
df.columns.values[2] = "momentum_score"

df=df.drop(['empty'], axis=1)

finalframe=df.drop(index=0)

finalframe['momentum_score'] =finalframe['momentum_score'].astype(float)

finalframe['rank']=finalframe['momentum_score'].rank(ascending=False)

finalframe['momentum_score'] =finalframe['momentum_score'].astype(float)

#finalframe.style.format({'momentum_score':"{:.2f}"})

Mylist = finalframe["symbol"].tolist()

finalframe.set_index('symbol',inplace=True)

final = pd.concat([df1, df2], axis=1, join='inner')
final.columns.values[3] = "tickers"
final.columns = final.columns.astype(str)

final.set_index('tickers', inplace=True)
Yes = final.reindex(Mylist)

rt = pd.concat([finalframe, Yes], axis=1, join='inner')

rt = rt.iloc[:, [2, 3, 4, 0, 1]]

#rt.style.format({'momentum_score':"{:.2f}"})

# get names of indexes for which column "momentum_score" has value > 40%
index_names = rt[(rt['momentum_score'] < 0.001)].index

# drop these given row indexes from dataFrame
rt.drop(index_names, inplace = True)

rt.reset_index(inplace=True)

My_final_list = rt["index"].tolist()

rt.set_index('index',inplace=True)

final_table_columns = My_final_list
final_table =stocks.drop(columns=[col for col in stocks if col not in final_table_columns])

for column in final_table.columns:
    final_table[column].to_csv('/content/momentum_tickers/' + column + '.csv')

In [9]:
from pathlib import Path
import pandas as pd
import stockstats
from stockstats import StockDataFrame as sdf
import glob
vola_table = Path('/content/momentum_tickers')

for file in vola_table.glob('*.JO.csv'): #create iteratable & iterate on it.
    vola_df = pd.read_csv(file)
    vola_df.columns.values[1] = "close"
    vola_df["20d_vol"] = vola_df["close"].pct_change().rolling(20).std()
    #vola_df = sdf.retype(vola_df)['close_90_roc']
    #vola_df = pd.DataFrame(vola_df)
    vola_df['date'] =  pd.to_datetime(vola_df['date']).dt.strftime('%Y-%m-%d')
    vola_df.set_index('date', inplace=True)
    stockstats_df = sdf.retype(vola_df)['close_90_roc']
    df = pd.DataFrame(stockstats_df)
    vola_file_name= file.parent.joinpath(f"/content/ranking_table/{file.stem}.csv")
    vola_df.to_csv(vola_file_name) #, index=None, encoding='utf-8-sig', decimal=',')

vola_table1 = Path('/content/ranking_table')

for file in vola_table1.glob('*.JO.csv'): #create iteratable & iterate on it.
    vola_df1 = pd.read_csv(file).tail(1)

    vola_df1['date'] =  pd.to_datetime(vola_df1['date']).dt.strftime('%Y-%m-%d')

    vola_df1.set_index('date', inplace=True)

    vola_file_name1= file.parent.joinpath(f"/content/momentum_stocks/{file.stem}.csv")

    vola_df1.to_csv(vola_file_name1)


# set search path and glob for files
# here we want to look for csv files in the input directory
path = '/content/'
files = glob.glob(path + '/momentum_stocks/*.csv')

# create empty list to store dataframes
li = []

# loop through list of files and read each one into a dataframe and append to list
for f in files:
    # get filename
    stock = os.path.basename(f)
    # read in csv
    temp_df = pd.read_csv(f)
    # create new column with filename
    temp_df['ticker'] = stock
    # data cleaning to remove the .csv
    temp_df['ticker'] = temp_df['ticker'].replace('.csv', '', regex=True)
    # append df to list
    li.append(temp_df)
    #print(f'Successfully created dataframe for {stock} with shape {temp_df.shape}')

# concatenate our list of dataframes into one!
df = pd.concat(li)

ranking_list =df.set_index('ticker').reindex(My_final_list).reset_index()

inv_vol = 1 / ranking_list["20d_vol"]
sum_inv_vol=inv_vol.sum()
vol_targets = inv_vol / sum_inv_vol
ranking_list["weight"] = vol_targets

ranking_list.set_index('ticker', inplace=True)

final_list = pd.concat([rt,ranking_list], axis=1, join='inner')

stockprice = final_list["close"] / 100
pos_weight = portfolio_value * final_list["weight"]
final_list["units"] = pos_weight / stockprice
final_list["ZAR"] = stockprice * final_list["units"]

In [10]:
from pathlib import Path
import pandas as pd

sma = Path('/content/ranking_table')

for file in sma.glob('*.JO.csv'): #create iteratable & iterate on it.
    sma = pd.read_csv(file)
    sma['date'] =  pd.to_datetime(sma['date']).dt.strftime('%Y-%m-%d')
    sma.set_index('date', inplace=True)

    '''Function'''

    sma['100d_sma'] = sma['close'].rolling(100).mean()

    #sma['index_sma']=index['Close'].rolling(200).mean()

    #sma['trend'] = np.where(sma['close'] > sma['100d_sma'], True, False)

    sma['index'] = index['Close']

    sma['index_sma']=index['Close'].rolling(200).mean()

    sma['trend'] = np.where(sma['close'] > sma['100d_sma'], True, False)

    sma['index_trend']= np.where(sma['index'] > sma['index_sma'], True, False)

    sma_file_name= file.parent.joinpath(f"/content/sma_data/{file.stem}.csv")

    sma.to_csv(sma_file_name)

buy_table = Path('/content/sma_data')

for file in buy_table.glob('*.JO.csv'): #create iteratable & iterate on it.
    buy_list_df = pd.read_csv(file).tail(1)

    buytable = file.parent.joinpath(f"/content/buy_list/{file.stem}.csv")

    buy_list_df.to_csv(buytable)

In [34]:
import pandas as pd
import glob

# set search path and glob for files
# here we want to look for csv files in the input directory
path1 = '/content/'
files1 = glob.glob(path1 + '/buy_list/*.csv')

# create empty list to store dataframes
li1 = []

# loop through list of files and read each one into a dataframe and append to list
for f1 in files1:
    # get filename
    stock1 = os.path.basename(f1)
    # read in csv
    temp_df1 = pd.read_csv(f1)
    # create new column with filename
    temp_df1['ticker'] = stock1
    # data cleaning to remove the .csv
    temp_df1['ticker'] = temp_df1['ticker'].replace('.csv', '', regex=True)
    # append df to list
    li1.append(temp_df1)
    #print(f'Successfully created dataframe for {stock1} with shape {temp_df1.shape}')

# concatenate our list of dataframes into one!
Buy = pd.concat(li1)

buying_list = Buy.set_index('ticker').reindex(My_final_list)#.reset_index()

final_buy_list = pd.concat([final_list, buying_list], axis=1, join='inner')

#define list of columns range
cols = [x for x in range(final_buy_list.shape[1])]

#remove second column in DataFrame
cols.remove(12)
cols.remove(13)
cols.remove(14)
cols.remove(15)
cols.remove(16)#view resulting DataFrame
final_buy_list2 =final_buy_list.iloc[:, cols]
final_buy_list2.info()

final_buy_list2 = final_buy_list2.loc[final_buy_list2.trend, :]

index_names1 = final_buy_list2[(final_buy_list2['close_90_roc'] >= 15)].index

# drop these given row indexes from dataFrame
final_buy_list2.drop(index_names1, inplace = True)
final_buy_list2.to_csv("portfolio_280723.csv")
final_buy_list2

<class 'pandas.core.frame.DataFrame'>
Index: 71 entries, TPF.JO to OAO.JO
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            71 non-null     object 
 1   symbol          71 non-null     object 
 2   ticker          71 non-null     object 
 3   momentum_score  71 non-null     float64
 4   rank            71 non-null     float64
 5   date            71 non-null     object 
 6   close           71 non-null     float64
 7   20d_vol         71 non-null     float64
 8   close_90_roc    67 non-null     float64
 9   weight          71 non-null     float64
 10  units           71 non-null     float64
 11  ZAR             71 non-null     float64
 12  100d_sma        64 non-null     float64
 13  index           71 non-null     float64
 14  index_sma       71 non-null     float64
 15  trend           71 non-null     bool   
 16  index_trend     71 non-null     bool   
dtypes: bool(2), float64(11), object(4

,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
TPF.JO,TRANSCEND RES PROP FD LD,TPF.XJSE,TPF,294.384088,1.0,2023-07-31,639.0,22.149721,-0.776398,0.000010,0.003214,0.020537,452.8714,78977.890625,75569.833262,True,True
SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.026048,4.0,2023-07-31,1880.0,0.014925,9.941520,0.015239,1.621127,30.477178,1879.9400,78977.890625,75569.833262,True,True
INL.JO,INVESTEC LTD,INL.XJSE,INL,1.019922,5.0,2023-07-31,11163.0,0.014061,14.398442,0.016176,0.289809,32.351400,10422.2000,78977.890625,75569.833262,True,True
ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,ART,0.966244,6.0,2023-07-31,1600.0,0.016424,8.769545,0.013848,1.730984,27.695751,1515.3200,78977.890625,75569.833262,True,True
INP.JO,INVESTEC PLC,INP.XJSE,INP,0.944983,7.0,2023-07-31,11200.0,0.013931,13.821138,0.016326,0.291538,32.652290,10509.7600,78977.890625,75569.833262,True,True
CLI.JO,CLIENTELE LTD,CLI.XJSE,CLI,0.628506,9.0,2023-07-31,1194.0,0.051441,13.714286,0.004421,0.740605,8.842820,1061.1400,78977.890625,75569.833262,True,True
QFH.JO,QUANTUM FOODS HLDGS LTD,QFH.XJSE,QFH,0.621167,10.0,2023-07-31,485.0,0.025594,7.777778,0.008887,3.664603,17.773324,455.0300,78977.890625,75569.833262,True,True
TRE.JO,TRENCOR LTD,TRE.XJSE,TRE,0.537838,12.0,2023-07-31,655.0,0.016858,5.645161,0.013492,4.119600,26.983381,647.5000,78977.890625,75569.833262,True,True
LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,LHC,0.411655,18.0,2023-07-31,2083.0,0.008727,11.152615,0.026063,2.502437,52.125769,2003.7800,78977.890625,75569.833262,True,True
DCP.JO,DIS-CHEM PHARMACIES LTD,DCP.XJSE,DCP,0.301979,29.0,2023-07-31,2570.0,0.022598,3.212851,0.010065,0.783258,20.129734,2463.6300,78977.890625,75569.833262,True,True


###List to Liquidate current positions

In [35]:
#read current portfolio from disc
sell=pd.read_csv("portfolio_280723.csv")
sell


sell_list = final_buy_list [~final_buy_list
 ["trend"]]
sell_list.info()
sell_list

#sell = pf

#sell_list[sell_list['index'].isin(sell)]

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, CML.JO to HAR.JO
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            39 non-null     object 
 1   symbol          39 non-null     object 
 2   ticker          39 non-null     object 
 3   momentum_score  39 non-null     float64
 4   rank            39 non-null     float64
 5   date            39 non-null     object 
 6   close           39 non-null     float64
 7   20d_vol         39 non-null     float64
 8   close_90_roc    35 non-null     float64
 9   weight          39 non-null     float64
 10  units           39 non-null     float64
 11  ZAR             39 non-null     float64
 12  Unnamed: 0      39 non-null     int64  
 13  date            39 non-null     object 
 14  close           39 non-null     float64
 15  20d_vol         39 non-null     float64
 16  close_90_roc    35 non-null     float64
 17  100d_sma        32 non-null     f

,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,...,Unnamed: 0,date,close,20d_vol,close_90_roc,100d_sma,index,index_sma,trend,index_trend
CML.JO,CORONATION FUND MNGRS LD,CML.XJSE,CML,0.924513,8.0,2023-07-31,3281.0,0.014011,NaN,0.016233,...,895,2023-07-31,3281.0,0.014011,NaN,NaN,78977.890625,75569.833262,False,True
TRU.JO,TRUWORTHS INT LTD,TRU.XJSE,TRU,0.510278,13.0,2023-07-31,7179.0,0.017240,NaN,0.013193,...,895,2023-07-31,7179.0,0.017240,NaN,NaN,78977.890625,75569.833262,False,True
RBP.JO,ROYAL BAFOKENG PLATINUM LTD,RBP.XJSE,RBP,0.507116,14.0,2023-07-31,12799.0,0.014423,-10.808362,0.015769,...,895,2023-07-31,12799.0,0.014423,-10.808362,13812.83,78977.890625,75569.833262,False,True
JSE.JO,JSE LTD,JSE.XJSE,JSE,0.482525,15.0,2023-07-31,9152.0,0.010932,-10.633727,0.020805,...,895,2023-07-31,9152.0,0.010932,-10.633727,NaN,78977.890625,75569.833262,False,True
BWN.JO,BALWIN PROPERTIES LTD,BWN.XJSE,BWN,0.469960,16.0,2023-07-31,250.0,0.008892,-18.032787,0.025577,...,895,2023-07-31,250.0,0.008892,-18.032787,280.16,78977.890625,75569.833262,False,True
ARI.JO,AFRICAN RAINBOW MIN LTD,ARI.XJSE,ARI,0.418183,17.0,2023-07-31,20075.0,0.018217,-13.807909,0.012485,...,895,2023-07-31,20075.0,0.018217,-13.807909,22028.09,78977.890625,75569.833262,False,True
GPL.JO,GRAND PARADE INV LTD,GPL.XJSE,GPL,0.382028,19.0,2023-07-31,298.0,0.022896,-10.510511,0.009934,...,895,2023-07-31,298.0,0.022896,-10.510511,317.94,78977.890625,75569.833262,False,True
YRK.JO,YORK TIMBER HOLDINGS LTD,YRK.XJSE,YRK,0.378707,20.0,2023-07-31,185.0,0.029260,-13.551402,0.007773,...,895,2023-07-31,185.0,0.029260,-13.551402,202.64,78977.890625,75569.833262,False,True
RMH.JO,RMB HOLDINGS LTD,RMH.XJSE,RMH,0.366062,21.0,2023-07-31,48.0,0.014148,-7.692308,0.016076,...,895,2023-07-31,48.0,0.014148,-7.692308,49.93,78977.890625,75569.833262,False,True
KAL.JO,KAAP AGRI LIMITED,KAL.XJSE,KAL,0.350401,22.0,2023-07-31,3768.0,0.010794,-5.563910,0.021072,...,895,2023-07-31,3768.0,0.010794,-5.563910,3803.62,78977.890625,75569.833262,False,True


In [36]:
dqs = sell_list ["ticker"].tolist()
dqs

['CML',
 'TRU',
 'RBP',
 'JSE',
 'BWN',
 'ARI',
 'GPL',
 'YRK',
 'RMH',
 'KAL',
 'HLM',
 'TBS',
 'WEZ',
 'APF',
 'BAW',
 'MPT',
 'REN',
 'CMH',
 'LEW',
 'NTC',
 'GML',
 'GSH',
 'AEG',
 'PPE',
 'LAB',
 'NTCP',
 'DRD',
 'PBG',
 'PPC',
 'IVT',
 'MFL',
 'GLN',
 'ISB',
 'MST',
 'VIS',
 '4SI',
 'QLT',
 'JBL',
 'HAR']

In [37]:
cpf=pd.read_csv("portfolio_140723.csv")
cpf.info()
cpf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      27 non-null     object 
 1   name            27 non-null     object 
 2   symbol          27 non-null     object 
 3   ticker          27 non-null     object 
 4   momentum_score  27 non-null     float64
 5   rank            27 non-null     float64
 6   date            27 non-null     object 
 7   close           27 non-null     float64
 8   20d_vol         27 non-null     float64
 9   close_90_roc    27 non-null     float64
 10  weight          27 non-null     float64
 11  units           27 non-null     float64
 12  ZAR             27 non-null     float64
 13  100d_sma        27 non-null     float64
 14  index           27 non-null     float64
 15  index_sma       27 non-null     float64
 16  trend           27 non-null     bool   
 17  index_trend     27 non-null     bool 

,Unnamed: 0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
0,VUN.JO,VUNANI LTD,VUN.XJSE,VUN,59.892215,1.0,2023-07-14,260.0,0.018176,-10.344828,0.013319,10.245601,26.638563,236.1500,77750.507812,74859.902324,True,True
1,SEB.JO,SEBATA HOLDINGS LTD,SEB.XJSE,SEB,3.075193,2.0,2023-07-14,220.0,0.109355,-12.000000,0.002214,2.012527,4.427560,188.4971,77750.507812,74859.902324,True,True
2,SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.544453,4.0,2023-07-14,1972.0,0.019070,9.799555,0.012695,1.287476,25.389029,1870.5900,77750.507812,74859.902324,True,True
3,TRE.JO,TRENCOR LTD,TRE.XJSE,TRE,0.923557,7.0,2023-07-14,670.0,0.035485,8.064516,0.006822,2.036480,13.644417,641.8500,77750.507812,74859.902324,True,True
4,ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,ART,0.613898,8.0,2023-07-14,1632.0,0.012478,9.898990,0.019402,2.377644,38.803154,1502.8500,77750.507812,74859.902324,True,True
5,REM.JO,REMGRO LTD,REM.XJSE,REM,0.606838,9.0,2023-07-14,15300.0,0.011028,10.365722,0.021952,0.286955,43.904071,13924.5900,77750.507812,74859.902324,True,True
6,LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,LHC,0.439935,16.0,2023-07-14,2049.0,0.010084,5.673027,0.024007,2.343246,48.013120,1989.7600,77750.507812,74859.902324,True,True
7,RLO.JO,REUNERT LTD,RLO.XJSE,RLO,0.274446,34.0,2023-07-14,5947.0,0.015125,0.337439,0.016006,0.538298,32.012604,5914.1000,77750.507812,74859.902324,True,True
8,ABG.JO,ABSA GROUP LIMITED,ABG.XJSE,ABG,0.271652,35.0,2023-07-14,17804.0,0.012469,-10.420126,0.019416,0.218105,38.831346,17379.3800,77750.507812,74859.902324,True,True
9,PPH.JO,PEPKOR HOLDINGS LTD,PPH.XJSE,PPH,0.232722,37.0,2023-07-14,1695.0,0.016413,-6.714364,0.014750,1.740358,29.499060,1679.2700,77750.507812,74859.902324,True,True


In [38]:
liquidate = cpf.loc[cpf['ticker'].isin (dqs)].reset_index (drop=True)
liquidate

,Unnamed: 0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
0,4SI.JO,4Sight Holdings Limited,4SI.XJSE,4SI,0.097897,54.0,2023-07-14,28.0,0.040222,-3.448276,0.006019,42.991592,12.037646,26.96,77750.507812,74859.902324,True,True
1,MFL.JO,METROFILE HOLDINGS LTD,MFL.XJSE,MFL,0.039159,63.0,2023-07-14,325.0,0.066643,5.177994,0.003633,2.235468,7.265271,313.42,77750.507812,74859.902324,True,True
2,JBL.JO,JUBILEE METALS GROUP PLC,JBL.XJSE,JBL,0.015929,70.0,2023-07-14,193.0,0.054717,-8.530806,0.004424,4.584854,8.848768,191.44,77750.507812,74859.902324,True,True


In [39]:
newpf = cpf[~cpf['ticker'].isin(liquidate['ticker'])]
newpf .info()
newpf

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 26
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      24 non-null     object 
 1   name            24 non-null     object 
 2   symbol          24 non-null     object 
 3   ticker          24 non-null     object 
 4   momentum_score  24 non-null     float64
 5   rank            24 non-null     float64
 6   date            24 non-null     object 
 7   close           24 non-null     float64
 8   20d_vol         24 non-null     float64
 9   close_90_roc    24 non-null     float64
 10  weight          24 non-null     float64
 11  units           24 non-null     float64
 12  ZAR             24 non-null     float64
 13  100d_sma        24 non-null     float64
 14  index           24 non-null     float64
 15  index_sma       24 non-null     float64
 16  trend           24 non-null     bool   
 17  index_trend     24 non-null     bool 

,Unnamed: 0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
0,VUN.JO,VUNANI LTD,VUN.XJSE,VUN,59.892215,1.0,2023-07-14,260.0,0.018176,-10.344828,0.013319,10.245601,26.638563,236.1500,77750.507812,74859.902324,True,True
1,SEB.JO,SEBATA HOLDINGS LTD,SEB.XJSE,SEB,3.075193,2.0,2023-07-14,220.0,0.109355,-12.000000,0.002214,2.012527,4.427560,188.4971,77750.507812,74859.902324,True,True
2,SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.544453,4.0,2023-07-14,1972.0,0.019070,9.799555,0.012695,1.287476,25.389029,1870.5900,77750.507812,74859.902324,True,True
3,TRE.JO,TRENCOR LTD,TRE.XJSE,TRE,0.923557,7.0,2023-07-14,670.0,0.035485,8.064516,0.006822,2.036480,13.644417,641.8500,77750.507812,74859.902324,True,True
4,ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,ART,0.613898,8.0,2023-07-14,1632.0,0.012478,9.898990,0.019402,2.377644,38.803154,1502.8500,77750.507812,74859.902324,True,True
5,REM.JO,REMGRO LTD,REM.XJSE,REM,0.606838,9.0,2023-07-14,15300.0,0.011028,10.365722,0.021952,0.286955,43.904071,13924.5900,77750.507812,74859.902324,True,True
6,LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,LHC,0.439935,16.0,2023-07-14,2049.0,0.010084,5.673027,0.024007,2.343246,48.013120,1989.7600,77750.507812,74859.902324,True,True
7,RLO.JO,REUNERT LTD,RLO.XJSE,RLO,0.274446,34.0,2023-07-14,5947.0,0.015125,0.337439,0.016006,0.538298,32.012604,5914.1000,77750.507812,74859.902324,True,True
8,ABG.JO,ABSA GROUP LIMITED,ABG.XJSE,ABG,0.271652,35.0,2023-07-14,17804.0,0.012469,-10.420126,0.019416,0.218105,38.831346,17379.3800,77750.507812,74859.902324,True,True
9,PPH.JO,PEPKOR HOLDINGS LTD,PPH.XJSE,PPH,0.232722,37.0,2023-07-14,1695.0,0.016413,-6.714364,0.014750,1.740358,29.499060,1679.2700,77750.507812,74859.902324,True,True


In [54]:
portfolio_value2=2054.76

In [56]:
final_weights = final_buy_list.drop(columns =["weight", "units", "ZAR"])
final_weights.reset_index()

,level_0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,Unnamed: 0,date,close,20d_vol,close_90_roc,100d_sma,index,index_sma,trend,index_trend
0,TPF.JO,TRANSCEND RES PROP FD LD,TPF.XJSE,TPF,294.384088,1.0,2023-07-31,639.0,22.149721,-0.776398,895,2023-07-31,639.0,22.149721,-0.776398,452.8714,78977.890625,75569.833262,True,True
1,TXT.JO,TEXTAINER GROUP HLDGS LTD,TXT.XJSE,TXT,1.657394,2.0,2023-07-31,71001.0,0.011747,25.665487,895,2023-07-31,71001.0,0.011747,25.665487,67536.1300,78977.890625,75569.833262,True,True
2,HCI.JO,HOSKEN CONS INV LTD,HCI.XJSE,HCI,1.233185,3.0,2023-07-31,22949.0,0.012076,21.423280,895,2023-07-31,22949.0,0.012076,21.423280,20561.7200,78977.890625,75569.833262,True,True
3,SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.026048,4.0,2023-07-31,1880.0,0.014925,9.941520,895,2023-07-31,1880.0,0.014925,9.941520,1879.9400,78977.890625,75569.833262,True,True
4,INL.JO,INVESTEC LTD,INL.XJSE,INL,1.019922,5.0,2023-07-31,11163.0,0.014061,14.398442,895,2023-07-31,11163.0,0.014061,14.398442,10422.2000,78977.890625,75569.833262,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,JBL.JO,JUBILEE METALS GROUP PLC,JBL.XJSE,JBL,0.006932,67.0,2023-07-31,170.0,0.038629,-8.602151,895,2023-07-31,170.0,0.038629,-8.602151,187.8800,78977.890625,75569.833262,False,True
67,SDO.JO,STADIO HOLDINGS LIMITED,SDO.XJSE,SDO,0.005469,68.0,2023-07-31,455.0,0.016048,5.813953,895,2023-07-31,455.0,0.016048,5.813953,450.1200,78977.890625,75569.833262,True,True
68,SYG.JO,SYGNIA LIMITED,SYG.XJSE,SYG,0.004363,69.0,2023-07-31,1850.0,0.028503,4.578858,895,2023-07-31,1850.0,0.028503,4.578858,1724.1700,78977.890625,75569.833262,True,True
69,HAR.JO,HARMONY GM CO LTD,HAR.XJSE,HAR,0.003595,70.0,2023-07-31,7695.0,0.021694,13.881900,895,2023-07-31,7695.0,0.021694,13.881900,8215.1700,78977.890625,75569.833262,False,True


In [57]:

'''
inv_vol = 1 / final_weights ["20d_vol"]
sum_inv_vol=inv_vol.sum()
vol_targets = inv_vol2 / sum_inv_vol
'''
final_weights["weight"] = vol_targets

#ranking_list.set_index('ticker', inplace=True)

#final_list = pd.concat([rt,ranking_list], axis=1, join='inner')

stockprice2 = final_weights["close"] / 100
pos_weight2 = portfolio_value2 * final_weights["weight"]
final_weights["units"] = pos_weight2 / stockprice2
final_weights["ZAR"] = stockprice2 * final_weights["units"]

ValueError: ignored

In [52]:
vol_targets2

,20d_vol,20d_vol
TPF.JO,0.000010,0.000010
TXT.JO,0.019361,0.019361
HCI.JO,0.018834,0.018834
SRE.JO,0.015239,0.015239
INL.JO,0.016176,0.016176
...,...,...
JBL.JO,0.005888,0.005888
SDO.JO,0.014173,0.014173
SYG.JO,0.007979,0.007979
HAR.JO,0.010484,0.010484


In [40]:
musthave = final_buy_list2 ["ticker"].tolist()
musthave

['TPF',
 'SRE',
 'INL',
 'ART',
 'INP',
 'CLI',
 'QFH',
 'TRE',
 'LHC',
 'DCP',
 'MTH',
 'AFH',
 'RDF',
 'SPG',
 'MTM',
 'ASC',
 'PPH',
 'PRX',
 'ATT',
 'MSP',
 'SSK',
 'SUR',
 'ABG',
 'ARL',
 'MTNZF',
 'SDO',
 'SYG']

In [21]:
dfnew = newpf[~newpf['ticker'].isin(final_buy_list2['ticker'])]
dfnew .info()
dfnew

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 26
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      11 non-null     object 
 1   name            11 non-null     object 
 2   symbol          11 non-null     object 
 3   ticker          11 non-null     object 
 4   momentum_score  11 non-null     float64
 5   rank            11 non-null     float64
 6   date            11 non-null     object 
 7   close           11 non-null     float64
 8   20d_vol         11 non-null     float64
 9   close_90_roc    11 non-null     float64
 10  weight          11 non-null     float64
 11  units           11 non-null     float64
 12  ZAR             11 non-null     float64
 13  100d_sma        11 non-null     float64
 14  index           11 non-null     float64
 15  index_sma       11 non-null     float64
 16  trend           11 non-null     bool   
 17  index_trend     11 non-null     bool 

,Unnamed: 0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
0,VUN.JO,VUNANI LTD,VUN.XJSE,VUN,59.892215,1.0,2023-07-14,260.0,0.018176,-10.344828,0.013319,10.245601,26.638563,236.1500,77750.507812,74859.902324,True,True
1,SEB.JO,SEBATA HOLDINGS LTD,SEB.XJSE,SEB,3.075193,2.0,2023-07-14,220.0,0.109355,-12.000000,0.002214,2.012527,4.427560,188.4971,77750.507812,74859.902324,True,True
5,REM.JO,REMGRO LTD,REM.XJSE,REM,0.606838,9.0,2023-07-14,15300.0,0.011028,10.365722,0.021952,0.286955,43.904071,13924.5900,77750.507812,74859.902324,True,True
7,RLO.JO,REUNERT LTD,RLO.XJSE,RLO,0.274446,34.0,2023-07-14,5947.0,0.015125,0.337439,0.016006,0.538298,32.012604,5914.1000,77750.507812,74859.902324,True,True
12,HDC.JO,HUDACO INDUSTRIES LTD,HDC.XJSE,HDC,0.217124,40.0,2023-07-14,16000.0,0.019365,-2.439024,0.012501,0.156264,25.002242,15699.8700,77750.507812,74859.902324,True,True
17,NWL.JO,NU-WORLD HLDGS LTD,NWL.XJSE,NWL,0.064964,58.0,2023-07-14,2450.0,0.021241,0.000000,0.011397,0.930382,22.794359,2418.3200,77750.507812,74859.902324,True,True
18,RBX.JO,RAUBEX GROUP LTD,RBX.XJSE,RBX,0.061587,60.0,2023-07-14,2601.0,0.039964,-8.253968,0.006058,0.465796,12.115343,2581.5900,77750.507812,74859.902324,True,True
19,DSBP.JO,DISCOVERY LTD B PREF,DSBP.XJSE,DSBP,0.046294,61.0,2023-07-14,9520.0,0.006791,-1.764524,0.035649,0.748927,71.297840,9373.9400,77750.507812,74859.902324,True,True
22,BVT.JO,BIDVEST LTD,BVT.XJSE,BVT,0.021902,66.0,2023-07-14,26103.0,0.011797,11.546515,0.020520,0.157227,41.040851,25231.1900,77750.507812,74859.902324,True,True
25,ENX.JO,ENX GROUP LIMITED,ENX.XJSE,ENX,0.014180,72.0,2023-07-14,700.0,0.060582,2.639296,0.003996,1.141724,7.992069,679.7200,77750.507812,74859.902324,True,True


In [22]:
remove = dfnew ["ticker"].tolist()
remove

['VUN', 'SEB', 'REM', 'RLO', 'HDC', 'NWL', 'RBX', 'DSBP', 'BVT', 'ENX', 'SNT']

In [23]:
now = newpf[~newpf['ticker'].isin(dfnew['ticker'])]
now .info()
now

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 2 to 24
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      13 non-null     object 
 1   name            13 non-null     object 
 2   symbol          13 non-null     object 
 3   ticker          13 non-null     object 
 4   momentum_score  13 non-null     float64
 5   rank            13 non-null     float64
 6   date            13 non-null     object 
 7   close           13 non-null     float64
 8   20d_vol         13 non-null     float64
 9   close_90_roc    13 non-null     float64
 10  weight          13 non-null     float64
 11  units           13 non-null     float64
 12  ZAR             13 non-null     float64
 13  100d_sma        13 non-null     float64
 14  index           13 non-null     float64
 15  index_sma       13 non-null     float64
 16  trend           13 non-null     bool   
 17  index_trend     13 non-null     bool 

,Unnamed: 0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
2,SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.544453,4.0,2023-07-14,1972.0,0.019070,9.799555,0.012695,1.287476,25.389029,1870.59,77750.507812,74859.902324,True,True
3,TRE.JO,TRENCOR LTD,TRE.XJSE,TRE,0.923557,7.0,2023-07-14,670.0,0.035485,8.064516,0.006822,2.036480,13.644417,641.85,77750.507812,74859.902324,True,True
4,ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,ART,0.613898,8.0,2023-07-14,1632.0,0.012478,9.898990,0.019402,2.377644,38.803154,1502.85,77750.507812,74859.902324,True,True
6,LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,LHC,0.439935,16.0,2023-07-14,2049.0,0.010084,5.673027,0.024007,2.343246,48.013120,1989.76,77750.507812,74859.902324,True,True
8,ABG.JO,ABSA GROUP LIMITED,ABG.XJSE,ABG,0.271652,35.0,2023-07-14,17804.0,0.012469,-10.420126,0.019416,0.218105,38.831346,17379.38,77750.507812,74859.902324,True,True
9,PPH.JO,PEPKOR HOLDINGS LTD,PPH.XJSE,PPH,0.232722,37.0,2023-07-14,1695.0,0.016413,-6.714364,0.014750,1.740358,29.499060,1679.27,77750.507812,74859.902324,True,True
10,ASC.JO,ASCENDIS HEALTH LTD,ASC.XJSE,ASC,0.223080,38.0,2023-07-14,64.0,0.030166,8.474576,0.008025,25.078702,16.050370,62.34,77750.507812,74859.902324,True,True
11,CLI.JO,CLIENTELE LTD,CLI.XJSE,CLI,0.218270,39.0,2023-07-14,1125.0,0.057045,8.173077,0.004244,0.754453,8.487591,1045.98,77750.507812,74859.902324,True,True
13,SUR.JO,SPUR CORPORATION LTD,SUR.XJSE,SUR,0.173518,46.0,2023-07-14,2360.0,0.008799,-0.631579,0.027513,2.331587,55.025458,2339.81,77750.507812,74859.902324,True,True
14,QFH.JO,QUANTUM FOODS HLDGS LTD,QFH.XJSE,QFH,0.144772,50.0,2023-07-14,481.0,0.027969,1.263158,0.008656,3.599027,17.311322,452.61,77750.507812,74859.902324,True,True


In [24]:
df_one = final_buy_list2.set_index(['ticker'])
df_one
df_two = now.set_index(['ticker'])
df_two

,Unnamed: 0,name,symbol,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
ticker,,,,,,,,,,,,,,,,,
SRE,SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,1.544453,4.0,2023-07-14,1972.0,0.019070,9.799555,0.012695,1.287476,25.389029,1870.59,77750.507812,74859.902324,True,True
TRE,TRE.JO,TRENCOR LTD,TRE.XJSE,0.923557,7.0,2023-07-14,670.0,0.035485,8.064516,0.006822,2.036480,13.644417,641.85,77750.507812,74859.902324,True,True
ART,ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,0.613898,8.0,2023-07-14,1632.0,0.012478,9.898990,0.019402,2.377644,38.803154,1502.85,77750.507812,74859.902324,True,True
LHC,LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,0.439935,16.0,2023-07-14,2049.0,0.010084,5.673027,0.024007,2.343246,48.013120,1989.76,77750.507812,74859.902324,True,True
ABG,ABG.JO,ABSA GROUP LIMITED,ABG.XJSE,0.271652,35.0,2023-07-14,17804.0,0.012469,-10.420126,0.019416,0.218105,38.831346,17379.38,77750.507812,74859.902324,True,True
PPH,PPH.JO,PEPKOR HOLDINGS LTD,PPH.XJSE,0.232722,37.0,2023-07-14,1695.0,0.016413,-6.714364,0.014750,1.740358,29.499060,1679.27,77750.507812,74859.902324,True,True
ASC,ASC.JO,ASCENDIS HEALTH LTD,ASC.XJSE,0.223080,38.0,2023-07-14,64.0,0.030166,8.474576,0.008025,25.078702,16.050370,62.34,77750.507812,74859.902324,True,True
CLI,CLI.JO,CLIENTELE LTD,CLI.XJSE,0.218270,39.0,2023-07-14,1125.0,0.057045,8.173077,0.004244,0.754453,8.487591,1045.98,77750.507812,74859.902324,True,True
SUR,SUR.JO,SPUR CORPORATION LTD,SUR.XJSE,0.173518,46.0,2023-07-14,2360.0,0.008799,-0.631579,0.027513,2.331587,55.025458,2339.81,77750.507812,74859.902324,True,True


In [25]:
buy = pd.concat([df_one,df_two], axis=1, join='inner')

In [26]:
buy

,name,symbol,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,...,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
ticker,,,,,,,,,,,,,,,,,,,,,
SRE,SIRIUS REAL ESTATE LTD,SRE.XJSE,1.026048,4.0,2023-07-31,1880.0,0.014925,9.941520,0.015239,1.621127,...,0.019070,9.799555,0.012695,1.287476,25.389029,1870.59,77750.507812,74859.902324,True,True
ART,ARGENT INDUSTRIAL LTD,ART.XJSE,0.966244,6.0,2023-07-31,1600.0,0.016424,8.769545,0.013848,1.730984,...,0.012478,9.898990,0.019402,2.377644,38.803154,1502.85,77750.507812,74859.902324,True,True
CLI,CLIENTELE LTD,CLI.XJSE,0.628506,9.0,2023-07-31,1194.0,0.051441,13.714286,0.004421,0.740605,...,0.057045,8.173077,0.004244,0.754453,8.487591,1045.98,77750.507812,74859.902324,True,True
QFH,QUANTUM FOODS HLDGS LTD,QFH.XJSE,0.621167,10.0,2023-07-31,485.0,0.025594,7.777778,0.008887,3.664603,...,0.027969,1.263158,0.008656,3.599027,17.311322,452.61,77750.507812,74859.902324,True,True
TRE,TRENCOR LTD,TRE.XJSE,0.537838,12.0,2023-07-31,655.0,0.016858,5.645161,0.013492,4.119600,...,0.035485,8.064516,0.006822,2.036480,13.644417,641.85,77750.507812,74859.902324,True,True
LHC,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,0.411655,18.0,2023-07-31,2083.0,0.008727,11.152615,0.026063,2.502437,...,0.010084,5.673027,0.024007,2.343246,48.013120,1989.76,77750.507812,74859.902324,True,True
SPG,SUPER GROUP LTD,SPG.XJSE,0.090921,48.0,2023-07-31,3433.0,0.013548,8.194138,0.016787,0.978000,...,0.009642,-7.778714,0.025107,1.529037,50.213560,3215.27,77750.507812,74859.902324,True,True
MTM,MOMENTUM MET HLDGS LTD,MTM.XJSE,0.072183,49.0,2023-07-31,1896.0,0.009045,6.937394,0.025145,2.652379,...,0.012406,-5.399898,0.019514,2.101670,39.028021,1816.35,77750.507812,74859.902324,True,True
ASC,ASCENDIS HEALTH LTD,ASC.XJSE,0.051144,53.0,2023-07-31,63.0,0.032385,10.526316,0.007023,22.295632,...,0.030166,8.474576,0.008025,25.078702,16.050370,62.34,77750.507812,74859.902324,True,True


In [27]:
df_three = final_buy_list.set_index(['ticker'])
df_three

,name,symbol,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,...,Unnamed: 0,date,close,20d_vol,close_90_roc,100d_sma,index,index_sma,trend,index_trend
ticker,,,,,,,,,,,,,,,,,,,,,
TPF,TRANSCEND RES PROP FD LD,TPF.XJSE,294.384088,1.0,2023-07-31,639.0,22.149721,-0.776398,0.000010,0.003214,...,895,2023-07-31,639.0,22.149721,-0.776398,452.8714,78977.890625,75569.833262,True,True
TXT,TEXTAINER GROUP HLDGS LTD,TXT.XJSE,1.657394,2.0,2023-07-31,71001.0,0.011747,25.665487,0.019361,0.054538,...,895,2023-07-31,71001.0,0.011747,25.665487,67536.1300,78977.890625,75569.833262,True,True
HCI,HOSKEN CONS INV LTD,HCI.XJSE,1.233185,3.0,2023-07-31,22949.0,0.012076,21.423280,0.018834,0.164134,...,895,2023-07-31,22949.0,0.012076,21.423280,20561.7200,78977.890625,75569.833262,True,True
SRE,SIRIUS REAL ESTATE LTD,SRE.XJSE,1.026048,4.0,2023-07-31,1880.0,0.014925,9.941520,0.015239,1.621127,...,895,2023-07-31,1880.0,0.014925,9.941520,1879.9400,78977.890625,75569.833262,True,True
INL,INVESTEC LTD,INL.XJSE,1.019922,5.0,2023-07-31,11163.0,0.014061,14.398442,0.016176,0.289809,...,895,2023-07-31,11163.0,0.014061,14.398442,10422.2000,78977.890625,75569.833262,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JBL,JUBILEE METALS GROUP PLC,JBL.XJSE,0.006932,67.0,2023-07-31,170.0,0.038629,-8.602151,0.005888,6.926919,...,895,2023-07-31,170.0,0.038629,-8.602151,187.8800,78977.890625,75569.833262,False,True
SDO,STADIO HOLDINGS LIMITED,SDO.XJSE,0.005469,68.0,2023-07-31,455.0,0.016048,5.813953,0.014173,6.229740,...,895,2023-07-31,455.0,0.016048,5.813953,450.1200,78977.890625,75569.833262,True,True
SYG,SYGNIA LIMITED,SYG.XJSE,0.004363,69.0,2023-07-31,1850.0,0.028503,4.578858,0.007979,0.862645,...,895,2023-07-31,1850.0,0.028503,4.578858,1724.1700,78977.890625,75569.833262,True,True


In [28]:
buyfinal = pd.concat([df_three,buy], axis=1, join='inner')
buyfinal

,name,symbol,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,...,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
ticker,,,,,,,,,,,,,,,,,,,,,
SRE,SIRIUS REAL ESTATE LTD,SRE.XJSE,1.026048,4.0,2023-07-31,1880.0,0.014925,9.941520,0.015239,1.621127,...,0.019070,9.799555,0.012695,1.287476,25.389029,1870.59,77750.507812,74859.902324,True,True
ART,ARGENT INDUSTRIAL LTD,ART.XJSE,0.966244,6.0,2023-07-31,1600.0,0.016424,8.769545,0.013848,1.730984,...,0.012478,9.898990,0.019402,2.377644,38.803154,1502.85,77750.507812,74859.902324,True,True
CLI,CLIENTELE LTD,CLI.XJSE,0.628506,9.0,2023-07-31,1194.0,0.051441,13.714286,0.004421,0.740605,...,0.057045,8.173077,0.004244,0.754453,8.487591,1045.98,77750.507812,74859.902324,True,True
QFH,QUANTUM FOODS HLDGS LTD,QFH.XJSE,0.621167,10.0,2023-07-31,485.0,0.025594,7.777778,0.008887,3.664603,...,0.027969,1.263158,0.008656,3.599027,17.311322,452.61,77750.507812,74859.902324,True,True
TRE,TRENCOR LTD,TRE.XJSE,0.537838,12.0,2023-07-31,655.0,0.016858,5.645161,0.013492,4.119600,...,0.035485,8.064516,0.006822,2.036480,13.644417,641.85,77750.507812,74859.902324,True,True
LHC,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,0.411655,18.0,2023-07-31,2083.0,0.008727,11.152615,0.026063,2.502437,...,0.010084,5.673027,0.024007,2.343246,48.013120,1989.76,77750.507812,74859.902324,True,True
SPG,SUPER GROUP LTD,SPG.XJSE,0.090921,48.0,2023-07-31,3433.0,0.013548,8.194138,0.016787,0.978000,...,0.009642,-7.778714,0.025107,1.529037,50.213560,3215.27,77750.507812,74859.902324,True,True
MTM,MOMENTUM MET HLDGS LTD,MTM.XJSE,0.072183,49.0,2023-07-31,1896.0,0.009045,6.937394,0.025145,2.652379,...,0.012406,-5.399898,0.019514,2.101670,39.028021,1816.35,77750.507812,74859.902324,True,True
ASC,ASCENDIS HEALTH LTD,ASC.XJSE,0.051144,53.0,2023-07-31,63.0,0.032385,10.526316,0.007023,22.295632,...,0.030166,8.474576,0.008025,25.078702,16.050370,62.34,77750.507812,74859.902324,True,True


In [30]:
pd.merge(final_buy_list, buy, on = "ticker", how = "outer")

,name_x,symbol_x,ticker,momentum_score_x,rank_x,date_x,close_x,20d_vol_x,close_90_roc_x,weight_x,...,20d_vol_y,close_90_roc_y,weight_y,units_y,ZAR_y,100d_sma_y,index_y,index_sma_y,trend_y,index_trend_y
0,TRANSCEND RES PROP FD LD,TPF.XJSE,TPF,294.384088,1.0,2023-07-31,639.0,22.149721,-0.776398,0.000010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEXTAINER GROUP HLDGS LTD,TXT.XJSE,TXT,1.657394,2.0,2023-07-31,71001.0,0.011747,25.665487,0.019361,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOSKEN CONS INV LTD,HCI.XJSE,HCI,1.233185,3.0,2023-07-31,22949.0,0.012076,21.423280,0.018834,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.026048,4.0,2023-07-31,1880.0,0.014925,9.941520,0.015239,...,0.01907,9.799555,0.012695,1.287476,25.389029,1870.59,77750.507812,74859.902324,True,True
4,INVESTEC LTD,INL.XJSE,INL,1.019922,5.0,2023-07-31,11163.0,0.014061,14.398442,0.016176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,JUBILEE METALS GROUP PLC,JBL.XJSE,JBL,0.006932,67.0,2023-07-31,170.0,0.038629,-8.602151,0.005888,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,STADIO HOLDINGS LIMITED,SDO.XJSE,SDO,0.005469,68.0,2023-07-31,455.0,0.016048,5.813953,0.014173,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,SYGNIA LIMITED,SYG.XJSE,SYG,0.004363,69.0,2023-07-31,1850.0,0.028503,4.578858,0.007979,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,HARMONY GM CO LTD,HAR.XJSE,HAR,0.003595,70.0,2023-07-31,7695.0,0.021694,13.881900,0.010484,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
final_buy_list

,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,...,Unnamed: 0,date,close,20d_vol,close_90_roc,100d_sma,index,index_sma,trend,index_trend
TPF.JO,TRANSCEND RES PROP FD LD,TPF.XJSE,TPF,294.384088,1.0,2023-07-31,639.0,22.149721,-0.776398,0.000010,...,895,2023-07-31,639.0,22.149721,-0.776398,452.8714,78977.890625,75569.833262,True,True
TXT.JO,TEXTAINER GROUP HLDGS LTD,TXT.XJSE,TXT,1.657394,2.0,2023-07-31,71001.0,0.011747,25.665487,0.019361,...,895,2023-07-31,71001.0,0.011747,25.665487,67536.1300,78977.890625,75569.833262,True,True
HCI.JO,HOSKEN CONS INV LTD,HCI.XJSE,HCI,1.233185,3.0,2023-07-31,22949.0,0.012076,21.423280,0.018834,...,895,2023-07-31,22949.0,0.012076,21.423280,20561.7200,78977.890625,75569.833262,True,True
SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.026048,4.0,2023-07-31,1880.0,0.014925,9.941520,0.015239,...,895,2023-07-31,1880.0,0.014925,9.941520,1879.9400,78977.890625,75569.833262,True,True
INL.JO,INVESTEC LTD,INL.XJSE,INL,1.019922,5.0,2023-07-31,11163.0,0.014061,14.398442,0.016176,...,895,2023-07-31,11163.0,0.014061,14.398442,10422.2000,78977.890625,75569.833262,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JBL.JO,JUBILEE METALS GROUP PLC,JBL.XJSE,JBL,0.006932,67.0,2023-07-31,170.0,0.038629,-8.602151,0.005888,...,895,2023-07-31,170.0,0.038629,-8.602151,187.8800,78977.890625,75569.833262,False,True
SDO.JO,STADIO HOLDINGS LIMITED,SDO.XJSE,SDO,0.005469,68.0,2023-07-31,455.0,0.016048,5.813953,0.014173,...,895,2023-07-31,455.0,0.016048,5.813953,450.1200,78977.890625,75569.833262,True,True
SYG.JO,SYGNIA LIMITED,SYG.XJSE,SYG,0.004363,69.0,2023-07-31,1850.0,0.028503,4.578858,0.007979,...,895,2023-07-31,1850.0,0.028503,4.578858,1724.1700,78977.890625,75569.833262,True,True
HAR.JO,HARMONY GM CO LTD,HAR.XJSE,HAR,0.003595,70.0,2023-07-31,7695.0,0.021694,13.881900,0.010484,...,895,2023-07-31,7695.0,0.021694,13.881900,8215.1700,78977.890625,75569.833262,False,True
